<a href="https://colab.research.google.com/github/Nikhitha-S-Pavan/mask_detection_yolov3/blob/main/mask_detection_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive/MyDrive/kaggle_dataset/

In [ ]:
!unzip archiveyolo.zip

In [ ]:
cd darknet


In [ ]:
#Configure yolo to use colab resources

!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!cat Makefile
!make

In [ ]:
! ./darknet

In [ ]:
!mv /content/drive/MyDrive/kaggle_dataset/yolo /content/darknet/

In [ ]:
cd yolo

In [ ]:
!mv labels/test/*.txt images/test/

In [ ]:
!mv labels/train/*.txt images/train/

In [ ]:
!mv labels/valid/*.txt images/valid/

In [ ]:
!sed -i 's/\/kaggle\/input\/maskdetection\/yolo/data/g' test.txt

In [ ]:
!sed -i 's/\/kaggle\/input\/maskdetection\/yolo/data/g' train.txt

In [ ]:
!sed -i 's/\/kaggle\/input\/maskdetection\/yolo/data/g' valid.txt

In [ ]:
!sed -i 's/\/kaggle\/input\/maskdetection\/yolo/data/g' valid.txt

In [ ]:
!sed -i 's/\/kaggle\/input\/maskdetection\/yolo\///g' obj.data

In [ ]:
cd ..


In [ ]:
!mv  yolo/images/ data/

In [ ]:
!mv yolo/train.txt /content/darknet/

In [ ]:
!mv yolo/test.txt /content/darknet/

In [ ]:
!mv yolo/valid.txt /content/darknet/

In [ ]:
!mv yolo/obj.data /content/darknet/

In [ ]:
!mv yolo/obj.names /content/darknet/

In [ ]:
pwd

In [ ]:

!sed -i 's/classes=20/classes=2/g' cfg/yolo-voc.2.0.cfg ## change classes=2 with your length of class
!sed -i 's/filters=125/filters=35/g' cfg/yolo-voc.2.0.cfg ## change filters=35 with (classes + 5) * num for me (2 + 5) * 5 = 35 
!cat cfg/yolo-voc.2.0.cfg

In [ ]:
##### train #######
!wget https://pjreddie.com/media/files/darknet19_448.conv.23

In [ ]:
!./darknet detector train obj.data cfg/yolo-voc.2.0.cfg darknet19_448.conv.23 -dont_show 0